# Defunctionalizing Dependent Type Families
###### (https://blog.poisson.chat/posts/2021-01-16-dependent-fcfs.html)

We can use type families to get an approximation of dependent types: a function `g` or a type family `G` may have a result whose type `F x` depends on a some agrument `x`:

In [5]:
:ext TypeFamilies
:ext PolyKinds
:ext DataKinds

import Data.Proxy
import Data.Kind

type family F (x :: Type) :: Type

g :: Proxy x -> F x
g = undefined

type family G (x :: Type) :: F x

What we'd like to be able to do is a type-level pattern match on GADTs indexed by type families.
We run into this issue when trying to defunctionalize a dependent function such as `G`.

In [12]:
type Exp a = a -> Type

data SG (x :: Type) :: Exp (F x)

-- an evaluation function mapping expressions to values
type family Eval (e :: Exp a) :: a

--type instance Eval (SG x) = G x

Eval has two arguments, the type `a`, which is implicit, and the expression `e` of type `Exp a`. Here `a` is inferred to be `F x`, but that has a type family which is not allowed.

A similar value level encoding does compile.

In [16]:
:ext GADTs

data Exp1 (a :: Type) where
  SG1 :: Proxy x -> Exp1 (F x)

eval :: Exp1 a -> a
eval (SG1 x) = g x

To get around this, we can encode type equality as a type

In [24]:
:ext TypeOperators
:ext UndecidableInstances

data a :=: b where
  Refl :: a :=: a
  
type family Rewrite (e :: a :=: b) (x :: a) :: b
type instance Rewrite Refl x = x

data SG2_ (x :: Type) (e :: F x :=: y) :: Exp y
type SG2 x = SG2_ x Refl

-- now we can define Eval on SG2_
type instance Eval (SG2_ x e) = Rewrite e (G x)